In [21]:
import numpy as np
import json
from konlpy.tag import Okt
from tensorflow import keras
from pyjosa.josa import Josa

### 문장(학습) 데이터 가져오기

In [22]:
f = open("file/sentence.txt", "r", encoding="utf-8")
t = f.read().split('\n')
text = ''
for i in t :
    text = text + i
print('코퍼스의 길이: ', len(text))

코퍼스의 길이:  881


### 데이터 정제 및 파일 저장

In [23]:
# 함수 1 : 딕셔너리에 데이터 정제
def make_dic(words):

    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 1: continue 
        if len(tmp) > 2: tmp = tmp[1:]

        set_word(dic, tmp)

        if word == ".":
            tmp = ["@"]
            continue
            
    return dic

# 함수 2 : 딕셔너리에 데이터 정제
def set_word(dic, s2):
    w1, w2 = s2 
    
    if not w1 in dic:
        dic[w1] = {}
        
    # print(dic)    
    if not w2 in dic[w1]:
        dic[w1][w2] = 1

In [24]:
# 딕셔너리에 데이터 정제
text = text.replace("…", "") 

# 형태소 분석
Okt=Okt() 
malist = Okt.pos(text, norm=True) 
words = []

for word in malist:
    if not word[1] in ["Punctuation"]:
        words.append(word[0])
    if word[0] == ".":
        words.append(word[0])
            # print(words)
dic = make_dic(words)
        
# 파일 저장
# dict_file = "file/sentence.json"
# word_list = "file/wordlist.json"
# json.dump(words, open(word_list,"w", encoding="utf-8"))
# json.dump(dic, open(dict_file,"w", encoding="utf-8"))

In [25]:
print(dic)

{'@': {'오늘': 1, '나': 1, '하늘': 1, '만나서': 1, '역': 1, '이': 1, '안녕하세요': 1, '제': 1, '저': 1, '너무': 1, '힘들다': 1, '어렵다': 1, '언제': 1, '시장': 1}, '오늘': {'은': 1}, '은': {'날씨': 1, '얼마': 1, '홍길동': 1}, '날씨': {'가': 1}, '가': {'좋네요': 1, '고파요': 1, '아파요': 1, '맑다': 1}, '좋네요': {'.': 1}, '나': {'는': 1}, '는': {'배': 1, '추워요': 1, '아파요': 1, '머리': 1, '0': 1, '인천': 1, '모델링': 1}, '배': {'가': 1}, '고파요': {'.': 1}, '추워요': {'.': 1}, '하늘': {'이': 1}, '이': {'파랗다': 1, '물건': 1, '제대로': 1}, '파랗다': {'.': 1}, '만나서': {'반가워요': 1}, '반가워요': {'.': 1}, '역': {'으로': 1}, '으로': {'가는': 1}, '가는': {'길이': 1}, '길이': {'어딘가': 1}, '어딘가': {'요': 1}, '요': {'.': 1}, '물건': {'은': 1}, '얼마': {'인가요': 1}, '인가요': {'.': 1}, '아파요': {'.': 1}, '머리': {'가': 1}, '안녕하세요': {'.': 1}, '제': {'이름': 1}, '이름': {'은': 1}, '홍길동': {'입니다': 1}, '입니다': {'.': 1}, '저': {'의': 1, '는': 1}, '의': {'나이': 1}, '나이': {'는': 1}, '0': {'살': 1}, '살': {'입니다': 1, '고': 1}, '인천': {'에': 1}, '에': {'살': 1, '가다': 1}, '고': {'있습니다': 1}, '있습니다': {'.': 1}, '모델링': {'을': 1}, '을': {'하고': 1}, '하고': {'있습니다': 1},

### 데이터 전처리

In [38]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록
chars = sorted(list(set(dic.keys()))) # 정렬
print('사용 문자의 수:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자
    
print(char_indices)
# print(indices_char)

사용 문자의 수: 57
{'0': 0, '@': 1, '가': 2, '가는': 3, '가다': 4, '값': 5, '고': 6, '고파요': 7, '길이': 8, '나': 9, '나올지': 10, '나이': 11, '날씨': 12, '너무': 13, '는': 14, '만나서': 15, '맑다': 16, '머리': 17, '모델링': 18, '모르겠습니다': 19, '물건': 20, '반가워요': 21, '배': 22, '살': 23, '시장': 24, '아파요': 25, '안녕하세요': 26, '안됩니다': 27, '어딘가': 28, '어렵다': 29, '언제': 30, '얼마': 31, '에': 32, '역': 33, '오늘': 34, '요': 35, '으로': 36, '은': 37, '을': 38, '의': 39, '이': 40, '이름': 41, '인가요': 42, '인천': 43, '입니다': 44, '있습니다': 45, '잘': 46, '저': 47, '제': 48, '제대로': 49, '좋네요': 50, '추워요': 51, '파랗다': 52, '하고': 53, '하늘': 54, '홍길동': 55, '힘들다': 56}


In [39]:
sentences = []
next_chars = []
maxlen = 5

for i in range(len(words)-1):
    sentences.append(words[i])
    next_chars.append(words[i + 1])

print('학습 구문 수:', len(sentences))

print(sentences)
# print(next_chars)

학습 구문 수: 425
['오늘', '은', '날씨', '가', '좋네요', '.', '나', '는', '배', '가', '고파요', '.', '나', '는', '추워요', '.', '하늘', '이', '파랗다', '.', '만나서', '반가워요', '.', '역', '으로', '가는', '길이', '어딘가', '요', '.', '이', '물건', '은', '얼마', '인가요', '.', '나', '는', '아파요', '.', '나', '는', '배', '가', '아파요', '.', '나', '는', '머리', '가', '아파요', '.', '안녕하세요', '.', '제', '이름', '은', '홍길동', '입니다', '.', '저', '의', '나이', '는', '0', '살', '입니다', '.', '저', '는', '인천', '에', '살', '고', '있습니다', '.', '나', '는', '모델링', '을', '하고', '있습니다', '.', '너무', '안됩니다', '.', '힘들다', '.', '어렵다', '.', '언제', '값', '이', '제대로', '나올지', '잘', '모르겠습니다', '.', '시장', '에', '가다', '.', '오늘', '은', '날씨', '가', '좋네요', '.', '나', '는', '배', '가', '고파요', '.', '나', '는', '추워요', '.', '하늘', '이', '파랗다', '.', '만나서', '반가워요', '.', '역', '으로', '가는', '길이', '어딘가', '요', '.', '이', '물건', '은', '얼마', '인가요', '.', '나', '는', '아파요', '.', '나', '는', '배', '가', '아파요', '.', '나', '는', '머리', '가', '아파요', '.', '안녕하세요', '.', '제', '이름', '은', '홍길동', '입니다', '.', '저', '의', '나이', '는', '0', '살', '입니다', '.', '저', '는', '인천', '에

### 모델 만들기

* train test 데이터 분할

In [28]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i in range(len(sentences)) :
    if sentences[i] == '.' or next_chars[i] == '.':
        continue
    #print(i, sentences[i], next_chars[i])    
    
    X[ i, len(sentences[i]), char_indices[sentences[i]] ] = 1 
    y[ i, char_indices[next_chars[i]] ] = 1

In [29]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [30]:
from tensorflow import keras
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from tensorflow.keras import optimizers

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [31]:
# 모델 구축하기(LSTM)
keras.backend.clear_session()
model1 = Sequential()
model1.add(LSTM(128, input_shape=(maxlen, len(chars))))
model1.add(Dense(len(chars)))
model1.add(Activation('softmax'))
optimizer = optimizers.RMSprop(learning_rate=0.01)
model1.compile(loss='categorical_crossentropy', optimizer=optimizer)
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               95232     
                                                                 
 dense (Dense)               (None, 57)                7353      
                                                                 
 activation (Activation)     (None, 57)                0         
                                                                 
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [32]:
model1.fit(x_train, y_train, batch_size=128, epochs=1000, verbose=0) 
y_pred = model1.predict(x_test, verbose=0)

print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', mean_squared_error(y_test, y_pred)**(1/2))
print('MAE: ',mean_absolute_error(y_test, y_pred)) 
print('R2: ', r2_score(y_test, y_pred)) 

MSE:  0.004962743
RMSE:  0.07044673728644277
MAE:  0.0147083495
R2:  0.31636559707158785


In [33]:
# model1.save("file/WTS_model.h5")

### 모델 테스트

In [34]:
# 예측 값 -> 문자 변환 함수
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [35]:
# 조사 함수
def josa_fuc(c, nextc): 
    try:
        return c + Josa.get_josa(c, nextc)
    except :
        return c + nextc    

In [41]:
def predict_mo(lst):
    result =''
    
    for i in lst:
        
        if i[-1] == '요' or i[-1] == '다':
            result += i
            continue
            
        x = np.zeros((1, 5, len(chars)))
        x[0, len(i), char_indices[i]] = 1.

        preds = model1.predict(x, verbose=0)[0]
        next_index = sample(preds)
        next_char = indices_char[next_index]
        
        y = josa_fuc(i, next_char)
        result += y + ' '
        
    return result

In [42]:
lst = ['오늘','날씨','맑다']

print(predict_mo(lst))

오늘은 날씨가 맑다
